In [1]:
from collections import Counter, defaultdict, deque, namedtuple
from copy import deepcopy
from datetime import datetime
import functools
import inspect
import json
import locale
import os
from pathlib import Path
import pickle
from pprint import pp, pprint
import re
import sys
import time
from typing import Dict, List

import numpy as np
import pandas as pd
import plotly.express as px
import jsonschema
from sklearn.model_selection import train_test_split
import xmltodict

from colorutils import Color

from dotenv import load_dotenv
from tqdm import tqdm

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl, process_to_jsonl
from aic_nlp_utils.pycfg import parse_pycfg_args, read_pycfg

from prompt_opt.dataset import cro_data
from prompt_opt.utils import is_valid_json

%load_ext autoreload
%autoreload 2

from prompt_opt.utils import *

sys.path.append("/home/drchajan/devel/python/FC/automated-fact-checking")

os.environ['VLLM_WORKER_MULTIPROC_METHOD']='spawn'
load_dotenv()

/home/drchajan/devel/python/FC/VENV/vllm/lib/python3.11/site-packages/aic_nlp_utils/json.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

# AlignScore

In [3]:
def sample_random(src_path, k, rng, lang="en", query_fmt="html", tag_document="document1", tag_claim="document2"):
    assert lang in ["en", "cs"]
    assert query_fmt in ["html", "json"]
    src_path = Path(src_path)
    raw_data = []
    for fname in src_path.iterdir():
        if fname.is_file():
            fdata = read_jsonl(fname)
            raw_data += list(rng.choice(fdata, k, replace=False))
    
    data = []
    for e in raw_data:
        if query_fmt == "html":
            if lang == "en":
                query = f"<{tag_document}>{e['document']}</{tag_document}>\n<{tag_claim}>{e['claim']}</{tag_claim}>"
            else:
                query = f"<{tag_document}>{e['translated_ces_document']}</{tag_document}>\n<{tag_claim}>{e['translated_ces_claim']}</{tag_claim}>"
        else:
            if lang == "en":
                query = {tag_document: e['document'], tag_claim: e['claim']}
            else:
                query = {tag_document: e['translated_ces_document'], tag_claim: e['translated_ces_claim']}
            
        answer = {"class": e["label"]}
        data.append({"query": query, "answer": answer}) 
    
    rng.permutation(data)
    pos_idxs = [i for i, e in enumerate(data) if e["answer"]["class"] == 1]
    neg_idxs = [i for i, e in enumerate(data) if e["answer"]["class"] == 0]
    
    # interleave positive and negative samples so every even-sized prefix is balanced
    interleaved_data = []
    for p, n in zip(pos_idxs, neg_idxs):
        interleaved_data.append(data[p])
        interleaved_data.append(data[n])
    
    labels = [e["answer"]["class"] for e in interleaved_data]
    
    print(f"# positive samples: {np.sum(labels)}/{len(interleaved_data)}")
    return interleaved_data

data = sample_random("/mnt/data/factcheck/AlignScore-data/Benchmarks/deepl_summac/val", k=100, rng=np.random.RandomState(1234), query_fmt="json")
write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/alignscore_bench_val_json_query_masked_tags.jsonl", data)

# data = sample_random("/mnt/data/factcheck/AlignScore-data/Benchmarks/deepl_summac/val", k=100, rng=np.random.RandomState(1234), query_fmt="json", tag_document="document", tag_claim="claim")
# write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/alignscore_bench_val_json_query_named_tags.jsonl", data)

# positive samples: 236/472


# Libgen

In [14]:
from libgen_api import LibgenSearch

s = LibgenSearch()
# results = s.search_author("Floyd")
results = s.search_title("music modes")
pp(results)
# results = s.search_title("Pride and Prejudice")
# item_to_download = results[0]
# download_links = s.resolve_download_links(item_to_download)
# pp(download_links)

[{'ID': '733677',
  'Author': 'Gioseffo Zarlino',
  'Title': 'On the Modes: Part Four of Le Istitutioni Harmoniche, 1558 (Music '
           'Theory Translation Series) (Pt. 4)',
  'Publisher': 'Yale Univ Pr',
  'Year': '1983',
  'Pages': '150',
  'Language': 'English',
  'Size': '3 Mb',
  'Extension': 'pdf',
  'Mirror_1': 'http://library.lol/main/2C5079BF31ADF249E9ADC34C1021307C',
  'Mirror_2': 'http://libgen.li/ads.php?md5=2C5079BF31ADF249E9ADC34C1021307C',
  'Mirror_3': 'https://library.bz/main/edit/2C5079BF31ADF249E9ADC34C1021307C'},
 {'ID': '2654496',
  'Author': 'Brent,Jeff',
  'Title': 'Modalogy: Scales, Modes & Chords: The Primordial Building Blocks '
           'of Music',
  'Publisher': 'Hal Leonard',
  'Year': '2011;2013',
  'Pages': '',
  'Language': 'English',
  'Size': '2 Mb',
  'Extension': 'epub',
  'Mirror_1': 'http://library.lol/main/FE37ECAA1E27A865393C62FB6FC2D662',
  'Mirror_2': 'http://libgen.li/ads.php?md5=FE37ECAA1E27A865393C62FB6FC2D662',
  'Mirror_3': 'https:/

# CRO

In [2]:
def get_paragraphs(recs):
    pars = []
    for r in recs:
        text = r["text"]
        pars += text.split('\n\n')
    return pars

def get_initial_spans(recs, n_pars=5, max_pars=None, seed=1234):
    if not max_pars:
        max_pars = n_pars
    txts = []
    for r in recs:
        text = r["ftext"]
        spans = text.split('\n\n')
        if len(spans) >= n_pars:
            if max_pars == "all":
                txts.append("\n\n".join(spans))
            else:
                txts.append("\n\n".join(spans[:max_pars]))
    np.random.RandomState(seed).shuffle(txts)
    return txts

## People

In [ ]:
txts = get_initial_spans(recs, n_pars=5)
len(txts)

103199

In [8]:
data = [{"query": t, "answer": None} for t in txts[20:30]]
write_jsonl("people_to_label.jsonl", data)

In [ ]:
data = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles_raw.jsonl")
len(data)

20

In [13]:
pf(data[2]["answer"])

{'Markéta Vaňková': ['kandidátka na brněnskou primátorku', 'členka ODS', 'advokátka vymáhající exekuce pro ostravský
dopravní podnik'], 'Andrej Babiš': ['premiér ČR', 'předseda ANO'], 'Petr Fiala': ['předseda ODS']}


This type of variable key objects (patternProperties in JSON schema) is not yet supported by ObjectAligner. Let's transform to list of dicts. Note names of key will give hints...

In [14]:
def transform_to_list(data):
    for e in data:
        e["answer"] = [{"name": k, "roles": v} for k, v in e["answer"].items()]

transform_to_list(data)

In [16]:
data[2]

{'query': 'Nezastupuji žádné kmotry, ODS se posunula, říká kandidátka na brněnskou primátorku Vaňková\n\nPřekvapivý obrat ve vývoji povolebních jednání v Brně označil premiér Andrej Babiš (ANO) za megapodraz. Jeho hnutí ANO totiž podle všeho zůstane v opozici, přestože volby vyhrálo – a primátorkou se pravděpodobně stane Markéta Vaňková z ODS.\n\n„ODS se vymezuje jako alternativa k hnutí ANO, zejména na celostátní úrovni... Na nižších úrovních je třeba vždy zvažovat konkrétní situaci, nicméně Brno je velké město a cítím od našich voličů, že preferují koalici bez ANO,“ vysvětluje Vaňková.\n\nNěkteří komentátoři ale připravovanou koalici ODS, KDU-ČSL, České pirátské strany (Piráti) a ČSSD nevítají s nadšením. „Nezastupuji žádné kmotry. Strana se posunula, což je naprosto zřejmé a občané to vidí. Líbí se mi, kam ODS pod vedením Petra Fialy směřuje,“ reaguje.\n\nPozornost vzbudil i fakt, že Vaňková v minulosti jako advokátka vymáhala exekuce pro ostravský dopravní podnik. „Zásadní je, že n

In [17]:
write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles.jsonl", data)

### V2
- hand fixed errors/changes in the original 20 V1 samples
- added 10 new samples (inicialised by model trained on V1)
- rearranged so we have reasonable balance of empty answers (no people) between the following splits:=
   - `trn=[0:8]`,
   - `dev=[8:16]`,
   - `tst=[16:30]`.
- assigned `id`s (corresponds to index in this set, starting from one) and `cro_id`s (zero-base index to original CRO data).

In [ ]:
# combine hand fixed with predicted (to hand fix them)
data = read_jsonl("data/labeled_datasets/people_roles_V2_pre.jsonl")
data += [{"query": e["query"], "answer": e["pred"]} for e in read_jsonl("data/labeled_datasets/people_roles_V2_predicted.jsonl")]
write_jsonl("data/labeled_datasets/people_roles_V2_fix_last_10.jsonl", data)

In [22]:
# now read all (hand) fixed and assign CRO_IDS
cro_idxs = [0, 1, 2, 3, 4, 5, 6, 16, 8, 9, 10, 7, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
data = read_jsonl("data/labeled_datasets/people_roles_V2_last_10_fixed.jsonl")
assert len(data) == len(cro_idxs)
for e, idx in zip(data, cro_idxs):
    e["cro_idx"] = idx
    assert e["query"] == txts[idx] # sanity check

In [41]:
def nempty(slice):
    return len(list(filter(lambda e: e["answer"] == [], slice)))

empty_answer = [e["answer"] == [] for e in data]

print("V1 distribution")
print("trn", nempty(data[0:4]))
print("dev", nempty(data[4:12]))
print("tst", nempty(data[12:20]))

print("V2 initial distribution")
print("trn", nempty(data[0:8]))
print("dev", nempty(data[8:16]))
print("tst", nempty(data[16:30]))

trn_dev, tst, trn_dev_empty_answer, _ = train_test_split(data, empty_answer, test_size=14, random_state=1234, stratify=empty_answer)
trn, dev = train_test_split(trn_dev, test_size=8, random_state=1234, stratify=trn_dev_empty_answer)
ndata = trn + dev + tst
print("V2 final distribution")
print("trn", nempty(ndata[0:8]))
print("dev", nempty(ndata[8:16]))
print("tst", nempty(ndata[16:30]))

for i, e in enumerate(ndata):
    e["id"] = i+1
    
write_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles_V2.jsonl", ndata)

V1 distribution
trn 0
dev 1
tst 1
V2 initial distribution
trn 1
dev 0
tst 6
V2 final distribution
trn 2
dev 2
tst 3


In [101]:
prompt = """**Query-to-Answer Transformation Instructions**

**Step 1: Identify Organization Entities**

* Read the query text carefully to identify organization entities including full names and abbreviations.

**Step 2: Format Entity Information**

* Create an object with a "name" property and a "abbreviation" property for each organization entity.
* Use the fullest possible name, i.e. the longest, not shortened, one. If the text contains only an abbreviation then sue empty string "".
* Use the abbreviation only if in the tect, otherwise use empty string: "".

**Step 3: Construct the Answer**

* Create an array of objects, where each object represents an organization entity with its name and abbreviation.
* Use JSON syntax to format the answer.

**Example Answer Element**

```json
{
  "name": "Organization Name, empty string if none in the text",
  "abbreviation": "Organization Abbreviation, empty string if none in the text"
}
```"""

with open("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/orgs/V1/bootstrap_prompt.jsonl", "w") as file:
    file.write(prompt)

In [104]:
# prepare to fix by hand
data = read_jsonl("data/labeled_datasets/orgs/V1/predicted.jsonl")
for e in data:
    e["answer"] = e["pred"]
    del e["pred"]
    
write_jsonl("data/labeled_datasets/orgs/V1/to_fix.jsonl", data)


data = read_jsonl("data/labeled_datasets/orgs/V1/predicted_ner_types.jsonl")
for e in data:
    e["answer"] = e["pred"]
    del e["pred"]
    
write_jsonl("data/labeled_datasets/orgs/V1/to_fix_ner_types.jsonl", data)

Annotation guidelines (updated after `data/labeled_datasets/orgs/V1/bootstrap_prompt_ner_types.md`):
* Create an object with a "name" property, an "abbreviation" property and a "type" for each organization entity.
* Use the fullest possible name, i.e. the longest, not shortened, one. If the text contains only an abbreviation then use empty string "".
* Use the abbreviation only if in the tect, otherwise use empty string: "".
* The type of an organization is one of the following (and nothing else):
  * `com`: Company - e.g., Apple, Microsoft, Tesla,
  * `gov`: Government Agency – e.g., NASA, FBI, United Nations; may be location name if it refers to government or an official body,
  * `edu`: Educational Institution – e.g., Harvard University, MIT,
  * `mil`: Military – e.g., US Army, 101st Airborne Division,
  * `art`: Art – e.g., Louvre Museum, The Beatles, Academy of American Poets,
  * `np`: Non-Profit Organization – e.g., Red Cross, WWF,
  * `sport`: Sports Team – e.g., Manchester United, Los Angeles Lakers,
  * `political`: Political Party – e.g., Democratic Party, Conservative Party,
  * `media`: Media Organization – e.g., BBC, The New York Times,
  * `fin`: Financial Institution – e.g., JPMorgan Chase, Goldman Sachs,
  * `health`: Healthcare Organization – e.g., Mayo Clinic, World Health Organization (WHO),
  * `religious`: Religious Organization – e.g., Vatican, Buddhist Sangha.

In [9]:
# add cro_idx
peopleV2 = read_jsonl("data/labeled_datasets/people_roles_V2.jsonl")
data = read_jsonl("data/labeled_datasets/orgs/V1/fixed.jsonl")
assert len(peopleV2) == len(data)
for d, p in zip(data, peopleV2):
    assert d["query"] == p["query"]
    d["cro_idx"] = p["cro_idx"]
write_jsonl("data/labeled_datasets/orgs_V1.jsonl", data)

## Locs

### V1
- similarly to `org_V1` reuses queries from `people_roles_V2`
- prediction-bootstraped 30 samples using modified prompt for people.

In [2]:
# prepare to fix by hand
data = read_jsonl("data/labeled_datasets/locs/V1/predicted_ner_types.jsonl")
for e in data:
    e["answer"] = e["pred"]
    del e["pred"]
    
write_jsonl("data/labeled_datasets/locs/V1/to_fix_ner_types.jsonl", data)

Annotation guidelines (updated after `data/labeled_datasets/locs/V1/bootstrap_prompt_ner_types.md`):
* Extract even mentions, e.g., 'ostravský dopravní podnik" => "Ostrava"
* EU is "gpe" 
* Create an object with a "name" property, an "abbreviation" property and a "type" for each location entity.
* Use the fullest possible name, i.e. the longest, not shortened, one (e.g., United States of America). If the text contains only an abbreviation then sue empty string "".
* Use the abbreviation (e.g., USA, NYC) only if in the tect, otherwise use empty string: "".
* The type of a location is one of the following (and nothing else):
  * `gpe`: Geo-Political Entity - e.g., France, Tokyo, California,
  * `loc`: Physical Location - e.g., Mount Everest, Sahara Desert, Amazon River,
  * `fac`: Facility - e.g., Eiffel Tower, Heathrow Airport,
  * `address`: Address - e.g., 1600 Pennsylvania Avenue, 221B Baker Street,
  * `region`: Regions - e.g., The Midwest, Silicon Valley, The Balkans,
  * `coords`: Coordinate Locations - e.g., 40.7128° N, 74.0060° W,
  * `celestial`: Celestial Bodies - e.g., Mars, Andromeda Galaxy, The Moon,


In [10]:
# add cro_idx
peopleV2 = read_jsonl("data/labeled_datasets/people_roles_V2.jsonl")
data = read_jsonl("data/labeled_datasets/locs/V1/fixed.jsonl")
assert len(peopleV2) == len(data)
for d, p in zip(data, peopleV2):
    assert d["query"] == p["query"]
    d["cro_idx"] = p["cro_idx"]
write_jsonl("data/labeled_datasets/locs_V1.jsonl", data)

## Events

In [3]:
data = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/events.jsonl")

In [8]:
len(data)

20

In [7]:
pf(data[0]["answer"])

[{'event': 'V únoru 2022 EU navrhuje přijetí více sankcí vůči ruským představitelům, kteří se podíleli na uznání
nezávislosti separatistických regionů na východě Ukrajiny.', 'people': [2374, 1784]}, {'event': 'Maďarsko zatím sankce
nepodpořilo.', 'people': []}, {'event': 'Viktor Orbán je vnímán jako spojenec ruského prezidenta Vladimira Putina.',
'people': [6301, 1600]}]


### V2

In [42]:
recs, kws = cro_data.import_cro("/mnt/data/cro/factcheck/v1/interim/cro_all.jsonl", prepend_date=True)
txts = get_initial_spans(recs, n_pars=6) # one more for the date

len(txts)

103199

In [53]:
dataset_people = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/people_roles_V2.jsonl")
# hand-fixed events V1 - add cro_idxs
events_fixed = read_jsonl("/home/drchajan/devel/python/FC/prompt_opt/data/labeled_datasets/events_V2_pre.jsonl")
cro_idxs = [0, 1, 2, 3, 4, 5, 6, 16, 8, 9, 10, 7, 11, 12, 13, 14, 15, 17, 18, 19]
assert len(events_fixed) == len(cro_idxs)
for e, idx in zip(events_fixed, cro_idxs):
    e["cro_idx"] = idx
    assert txts[idx].splitlines()[0] in e["query"] # sanity check - compare at least the first line (date)
    # print(e["query"])
    # print("------------------")
    # print(txts[idx])
    # print("=====================================================\n")

In [75]:
def update_queries(txts, events_fixed, dataset_people):
    cro2people = {p["cro_idx"]: p for p in dataset_people}
    rng = np.random.RandomState(1234)
    queries = []
    for event in events_fixed:
        cro_idx = event["cro_idx"]
        txt = txts[cro_idx]
        date_ = txt.splitlines()[0]
        txt = "\n".join(txt.splitlines()[2:])
        query = "<date>" + date_ + "</date>\n"
        query += "<text>" + txt.strip() + "</text>\n"

        people = cro2people[cro_idx]["answer"]
        ids = rng.choice(10000, size=len(people), replace=False)
        query += "<people>"
        for person, id_ in zip(people, ids):
            name = person["name"]
            roles = person["roles"]
            query += f'<person id={id_} name="{name}">\n'
            for role in roles:
                query += "   <role>" + role + "</role>\n"
            query += "</person>\n"
        query += "</people>\n"
        if query != event["query"]:
            print(query)
            print("------------------")
            print(event["query"])
            print("=====================================================\n")
    
        queries.append(query)
    return queries


# now fix queries in the first 20 from events_V1 due to changes in people_roles_V2
# outputs differences due to changes between people_roles V1 and V2, seems fine
queries_first20 = update_queries(txts, events_fixed, dataset_people)

for e, q in zip(events_fixed, queries_first20):
    e["query"] = q


In [85]:
def prepare_queries(txts, cro_idxs, dataset_people, rng):
    cro2people = {p["cro_idx"]: p for p in dataset_people}
    rng = np.random.RandomState(1234)
    queries = []
    for cro_idx, people in zip(cro_idxs, dataset_people):
        txt = txts[cro_idx]
        date_ = txt.splitlines()[0]
        txt = "\n".join(txt.splitlines()[2:])
        query = "<date>" + date_ + "</date>\n"
        query += "<text>" + txt.strip() + "</text>\n"

        people = cro2people[cro_idx]["answer"]
        ids = rng.choice(10000, size=len(people), replace=False)
        query += "<people>"
        for person, id_ in zip(people, ids):
            name = person["name"]
            roles = person["roles"]
            query += f'<person id={id_} name="{name}">\n'
            for role in roles:
                query += "   <role>" + role + "</role>\n"
            query += "</person>\n"
        query += "</people>\n"
        queries.append(query)
    return queries

queries_new10 = prepare_queries(txts, list(range(20, 30)), dataset_people, rng=np.random.RandomState(1235))


In [86]:
# prepare the queries for annotation
data = [{"query": q, "answer": ["LABEL_THIS"]} for q in queries_new10]
write_jsonl("data/labeled_datasets/events_V2_annotate_last_10.jsonl", data)

In [87]:
events_fixed

[{'id': 1,
  'query': '<date>Út, 22 úno 2022 16:21:00</date>\n<text>Evropská unie uvalí sankce na ruské představitele a banky. Maďarsko je zdrženlivé\n\nEvropská unie navrhuje přijetí více sankcí vůči ruským představitelům, kteří se podíleli na uznání nezávislosti separatistických regionů na východě Ukrajiny. Ve společném prohlášení to uvedli předsedkyně Evropské komise Ursula von der Leyenová a šéf Evropské rady Charles Michel. Maďarsko jako jediná členská země zatím nedalo najevo jasnou podporu zvažované podobě postihů. Diplomaté se znovu sejdou večer, kdy by mohlo padnout konečné rozhodnutí.\n\nNavržené postihy, jejichž přijetí dopoledne předběžně podpořila většina velvyslanců členských zemí, se mají vztahovat na více než tři stovky ruských představitelů, sdělil novinářům diplomatický zdroj.\n\nEU však musí o zavedení sankcí rozhodovat jednomyslně a Maďarsko, jehož premiér Viktor Orbán je vnímán jako spojenec ruského prezidenta Vladimira Putina, podle diplomatů dosud bez přímého poz

In [89]:
# now combine all we have and prepare it for hand-fixing (check first twenty and fix last 10)
data = read_jsonl("data/labeled_datasets/events_V2_predicted_last_10.jsonl")
for e in data:
    e["answer"] = e["pred"]
    
write_jsonl("data/labeled_datasets/events_V2_to_fix.jsonl", events_fixed+data)

In [94]:
# finally fix cro_idx attributes and save the final dataset
cro_idxs = [0, 1, 2, 3, 4, 5, 6, 16, 8, 9, 10, 7, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
data = read_jsonl("data/labeled_datasets/events_V2_fixed.jsonl")
assert len(data) == len(cro_idxs)
for e, idx in zip(data, cro_idxs):
    e["cro_idx"] = idx
    print(e["query"])
    print("------------------")
    print(txts[idx])
    print("=====================================================\n")
write_jsonl("data/labeled_datasets/events_V2.jsonl", data)

<date>Út, 22 úno 2022 16:21:00</date>
<text>Evropská unie uvalí sankce na ruské představitele a banky. Maďarsko je zdrženlivé

Evropská unie navrhuje přijetí více sankcí vůči ruským představitelům, kteří se podíleli na uznání nezávislosti separatistických regionů na východě Ukrajiny. Ve společném prohlášení to uvedli předsedkyně Evropské komise Ursula von der Leyenová a šéf Evropské rady Charles Michel. Maďarsko jako jediná členská země zatím nedalo najevo jasnou podporu zvažované podobě postihů. Diplomaté se znovu sejdou večer, kdy by mohlo padnout konečné rozhodnutí.

Navržené postihy, jejichž přijetí dopoledne předběžně podpořila většina velvyslanců členských zemí, se mají vztahovat na více než tři stovky ruských představitelů, sdělil novinářům diplomatický zdroj.

EU však musí o zavedení sankcí rozhodovat jednomyslně a Maďarsko, jehož premiér Viktor Orbán je vnímán jako spojenec ruského prezidenta Vladimira Putina, podle diplomatů dosud bez přímého pozemního útoku nehodlalo sankce 

Annotation guidelines updates:
- vyhodit casova urceni, - ziskame je v dalsim kroku...
  - nevyhazovat je, pokud jsou dulezita pro samotnou udalost (viz data u navstevy Zemana)
  - preferovat (prevadet na) absolutni: misto pred 4 lety, v roce 2014
- vsechna kvantifikovana tvrzeni (eventu): vysledky zapasu, uroky, pocty lidi.... Obecne asi vse, co by pomohlo s overovanim.
- atribuce soucasti tvrzeni
- ignorovat background viz vstup USA do 1. svetove valky
- pokud to neni nezbytne nepouzivat pocitove "tesi se na zavod".

For next versions:
- pridat lokace, organizace vcetne zkratek,  vzdy nejdelsi verze (Český rozhlas Radiožurnál, ne jen Radiožurnál)
- indikator budouci moznosti (zřejmě se zvýší úrokové sazby)
- atribuce v dalsich verzich do struktury (id pro lidi, organizace a staty se musi lisit!)
- subevents/doplnkove info: jedna uroven hierarchie, sem napriklad detailni cisla, pokud je jich vice

### V3
- adds organizations and locations
- adds future indicator: predictions/plans (zřejmě se zvýší úrokové sazby)
- attributions for loc and org

In [170]:
eventsV2 = read_jsonl("data/labeled_datasets/events_V2.jsonl")
peopleV2 = read_jsonl("data/labeled_datasets/people_roles_V2.jsonl")
orgsV1 = read_jsonl("data/labeled_datasets/orgs_V1.jsonl")
locsV1 = read_jsonl("data/labeled_datasets/locs_V1.jsonl")
assert len(eventsV2) == len(peopleV2) == len(orgsV1) == len(locsV1)

In [171]:
print(eventsV2[0]["query"].splitlines()[10])

<people><person id=2374 name="Ursula von der Leyenová">


In [172]:
def extend_query(events, locs, orgs, rng):
    events = deepcopy(events)
    locs = deepcopy(locs)
    orgs = deepcopy(orgs)
    # print("extend_query(): id=", events["id"])
    cro_idx = events["cro_idx"]
    cro2locs = {e["cro_idx"]: e for e in locs}
    cro2orgs = {e["cro_idx"]: e for e in orgs}
    locs = cro2locs[cro_idx]["answer"]
    orgs = cro2orgs[cro_idx]["answer"]
    xml_string = re.sub(r' id=(\d+)', r' id="\1"', events["query"])
    root = xmltodict.parse(f'<root>{xml_string}</root>')
    x = root["root"]
    
    # pprint(locs)
    if x["people"] is None:
        x["people"] = {'person': []}
        all_ids = set()
    else:
        if isinstance(x["people"]["person"], dict):
            x["people"]["person"] = [x["people"]["person"]]
        all_ids = set([e['@id'] for e in x["people"]["person"]])
        
    ids = set([str(e) for e in rng.choice(10000, size=len(locs), replace=False)])
    while len(all_ids.intersection(ids)) > 0:
        ids = set([str(e) for e in rng.choice(10000, size=len(locs), replace=False)])
        
    nlocs = []
    for loc, id_ in zip(locs, ids):
        nloc = {"@id": id_}
        nloc.update({f"@{k}": v for k, v in loc.items()})
        nlocs.append(nloc) # prepend id
    all_ids.update(ids)
    # pprint(x["people"])
    # pprint(nlocs)
    
    ids = set([str(e) for e in rng.choice(10000, size=len(orgs), replace=False)])
    while len(all_ids.intersection(ids)) > 0:
        ids = set([str(e) for e in rng.choice(10000, size=len(orgs), replace=False)])
        
    # pprint(ids)
    
    norgs = []
    for org, id_ in zip(orgs, ids):
        norg = {"@id": id_}
        norg.update({f"@{k}": v for k, v in org.items()})
        norgs.append(norg) # prepend id
    
        
    x["locations"] = {'loc': nlocs}
    x["organizations"] = {'org': norgs}
    query = xmltodict.unparse(root, pretty=True, indent="|")
    # now cut root and fix formatting
    query = '\n'.join(line[1:].replace('|', ' ') if line.startswith('|') else line for line in query.splitlines()[2:-1])
    # pprint(query)
    return query

idx = 10
ret = extend_query(eventsV2[idx], locsV1, orgsV1, np.random.RandomState(1234))
print(ret)

<date>Ne, 09 čec 2017 07:41:00</date>
<text>Dlouhodobé smlouvy jako v NHL? V Česku nemožné, nechtěli by je kluby ani hráči

Léto je v hokejovém světě mimo jiné obdobím uzavírání kontraktů. Třeba Carey Price nebo Connor McDavid mají v kapse nové, stamilionové smlouvy na celých osm let! O tom si můžou hráči v českém hokeji nechat jen zdát.

V roce 2025 bude silnice určitě křižovat více vozů s autopilotem. Solární energie bude možná všude na světě levnější, než ta z uhlí. A pravděpodobně se opět zrychlí tání ledovců. Útočník Connor McDavid ovšem v té době nejspíš bude stále hráčem Edmontonu.

V českých arénách na extraligovém hokeji něco takového není možné. Tady hráč podepíše během osmi let nový kontrakt hned několikrát.

Jágr chystá přesun do Evropy. Nabídky z NHL byly degradační, říká Marian Jelínek</text>
<people>
 <person id="2540" name="Carey Price">
  <role>útočník hokejového klubu Edmonton</role>
 </person>
 <person id="1907" name="Connor McDavid">
  <role>hokejový hráč s nově pod

In [173]:
def prepare_answer(events):
    answer = events["answer"]
    for e in answer:
        e["locations"] = []
        e["orgs"] = []
        e["attributions"] = []
        e["future"] = False
    return answer

# idx = 5
# prepare_answer(eventsV2[idx])

In [174]:
eventsV2[0]

{'id': 1,
 'query': '<date>Út, 22 úno 2022 16:21:00</date>\n<text>Evropská unie uvalí sankce na ruské představitele a banky. Maďarsko je zdrženlivé\n\nEvropská unie navrhuje přijetí více sankcí vůči ruským představitelům, kteří se podíleli na uznání nezávislosti separatistických regionů na východě Ukrajiny. Ve společném prohlášení to uvedli předsedkyně Evropské komise Ursula von der Leyenová a šéf Evropské rady Charles Michel. Maďarsko jako jediná členská země zatím nedalo najevo jasnou podporu zvažované podobě postihů. Diplomaté se znovu sejdou večer, kdy by mohlo padnout konečné rozhodnutí.\n\nNavržené postihy, jejichž přijetí dopoledne předběžně podpořila většina velvyslanců členských zemí, se mají vztahovat na více než tři stovky ruských představitelů, sdělil novinářům diplomatický zdroj.\n\nEU však musí o zavedení sankcí rozhodovat jednomyslně a Maďarsko, jehož premiér Viktor Orbán je vnímán jako spojenec ruského prezidenta Vladimira Putina, podle diplomatů dosud bez přímého pozem

In [175]:
# prepare to fix annotations by hand
data = [{"query": extend_query(e, locsV1, orgsV1, np.random.RandomState(1236)), "answer": prepare_answer(e)} for e in eventsV2]    
write_jsonl("data/labeled_datasets/events/V3/to_fix.jsonl", data)

In [185]:
# fix cro_idx, same order as people_V2!!!!!
eventsV3_fixed = read_jsonl("data/labeled_datasets/events/V3/fixed.jsonl")
eventsV2 = read_jsonl("data/labeled_datasets/events_V2.jsonl")
peopleV2 = read_jsonl("data/labeled_datasets/people_roles_V2.jsonl")

for v3, v2 in zip(eventsV3_fixed, eventsV2):
    v3["cro_idx"] = v2["cro_idx"]

cro2ev3_fixed = {e["cro_idx"]: e for e in eventsV3_fixed}
data = []
for p in peopleV2:
    d = cro2ev3_fixed[p["cro_idx"]]
    d["id"] = p["id"]
    data.append(d)
    

write_jsonl("data/labeled_datasets/events_V3.jsonl", data)

Annotation guidelines updates:
- attribution: put each source of the event (people and/or organization) if the same happens to be part of the event, add him/her/it to the people/organizations; only the source closest to the original source mentioned
- sport teams: if not clear add information on sport/representation ("volejbalová reprezentace České republiky")

TODO next:
- evaluate and decide how to deal with organization/location overlaps
- get new versions of people, orgs and locs from here! Some fixes happened!

#### Partial
Extracting people, locations, organizations, etc separatelly...

In [9]:
data = read_jsonl("data/labeled_datasets/events_V3.jsonl")

In [19]:
def transform_sample(sample, target, tags, target_element=None):
    target_element = target_element if target_element else target
    t = xmltodict.parse("<doc>" + sample["query"] + "</doc>")["doc"]
    events = sample["answer"]
    t['events'] = {"event": [{'@id': f'{i+1}','@text': e["event"]} for i, e in enumerate(events)]}
    ret = {}
    for e in t:
        if e in tags:
            ret[e] = t[e]
    # pprint(t)
    # pprint(event_xml)
    xml_string = xmltodict.unparse({'doc': ret}, pretty=True, indent='|')
    query = '\n'.join(line[1:].replace('|', ' ') if line.startswith('|') else line for line in xml_string.splitlines()[2:-1])
    # print(sample["answer"])
    answer = {target: [{"event_id": i+1, target_element: ev[target]} for i, ev in enumerate(sample["answer"])]}
    nsample = deepcopy(sample)
    nsample["query"] = query
    nsample["answer"] = answer
    return nsample
    
    

write_jsonl("data/labeled_datasets/events_V3_people.jsonl",
            [transform_sample(e, target="people", tags=["date", "text", "people", "events"]) for e in data])

write_jsonl("data/labeled_datasets/events_V3_locations.jsonl",
            [transform_sample(e, target="locations", tags=["date", "text", "locations", "events"]) for e in data])

write_jsonl("data/labeled_datasets/events_V3_orgs.jsonl",
            [transform_sample(e, target="orgs", tags=["date", "text", "organizations", "events"]) for e in data])

write_jsonl("data/labeled_datasets/events_V3_attributions.jsonl",
            [transform_sample(e, target="attributions", tags=["date", "text", "people", "organizations", "events"], target_element="attribution") for e in data])


In [35]:
data[0].keys()

dict_keys(['id', 'query', 'answer', 'cro_idx'])

In [17]:
data2 = read_jsonl("data/labeled_datasets/events_V3_orgs.jsonl")

In [18]:
print(data2[3]["query"])

<date>Po, 24 pro 2018 14:45:00</date>
<text>V Izraeli rozpustili parlament, předčasné volby plánují na duben

Izraelská vládní koalice se shodla na rozpuštění parlamentu. Předčasné volby se uskuteční příští rok v dubnu. Na twitteru to v pondělí napsal mluvčí premiéra Benjamina Netanjahua. Za rozhodnutím stojí koaliční krize, kterou způsobil zákon o výjimkách z povinné vojenské služby pro ultraortodoxní židy. Netanjahu má ve 120členném parlamentu těsnou většinu 61 hlasů.

„Vzhledem k národní a rozpočtové zodpovědnosti se lídři koaličních stran jednomyslně dohodli na rozpuštění Knesetu a na nových volbách na začátku dubna,“ uvedly strany ve svém společném prohlášení.

Podle listu The Times of Israel rovněž zdůraznily, že žádná ze stran vládu neopustí a že „partnerství v Knesetu a ve vládě bude pokračovat i během voleb“. Podle izraelských zákonů se příští volby měly konat až v listopadu 2019.

Ultraortodoxní židé tvoří deset až 12 procent izraelské populace. Ti, kteří se věnují náboženské

### V4
- adds hierarchy for events, fixes from V3 based on model predictions
TODO FIX:
- s30: remove CNB from locations

In [11]:
data = read_jsonl("data/labeled_datasets/events/V4/events_V3_fixed_all.jsonl")

In [27]:
def prepare_for_subevent_annotation(data):
    data = deepcopy(data)
    for s in data:
        for event in s["answer"]:
            items = list(event.items())
            items.insert(1, ("subevents", []))
            event.clear()
            event.update(items)
            
    return data 
        
data2 = prepare_for_subevent_annotation(data)
write_jsonl("data/labeled_datasets/events/V4/events_V4_to_fix.jsonl", data2)

In [38]:
def export_events_only(data):
    data = deepcopy(data)
    for s in data:
        s["answer"] = [{"event": e["event"], "subevents": e["subevents"]} for e in s["answer"]]
    return data
    
    
data = read_jsonl("data/labeled_datasets/events/V4/events_V4_fixed.jsonl")
data2 = export_events_only(data)
write_jsonl("data/labeled_datasets/events_V4_events.jsonl", data2)

In [40]:
data2[0]["answer"]

[{'event': 'Erdogan po čistkách převzal kontrolu nad tureckou armádou a jmenoval do vedení protizápadní nacionalisty.',
  'subevents': []},
 {'event': 'Erdogan se zúčastnil připomínky výročí smrti Mustafy Kemala Atatürka.',
  'subevents': [{'event': 'Erdogan kráčel za vojáky nesoucími věnec se stuhou s motivy turecké vlajky.'},
   {'event': 'Erdogan se v mauzoleu v centru Ankary poklonil u Atatürkova sarkofágu.'}]}]

In [45]:
schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "event": {"type": "string"},
            "subevents": {
                "type": "array",
                "items": {"type": "object", "properties": {"event": {"type": "string"}}},
            },
        },
        "required": ["event", "subevents"],
    },
}

for s in data2:
    jsonschema.validate(s["answer"], schema)

## Event Temporal Validity

### Bootstrap
- just rearange newest event data we have (incomplete V4 dataet)

In [55]:
eventsV4 = read_jsonl("data/labeled_datasets/events_V4_events.jsonl")

In [56]:
eventsV4[10]

{'id': 11,
 'query': '<date>So, 13 bře 2021 19:18:00</date>\n<text>Fotbalisté Baníku rozstříleli Příbram 5:0, Opava porazila gólem z penalty Pardubice\n\nFotbalisté Příbrami utrpěli ve 23. kole první ligy debakl 0:5 v Ostravě a na posledním místě tabulky ztrácejí už tři body na předposlední Opavu. Slezané totiž porazili nováčka z Pardubic 1:0. Zlín zvítězil nad Českými Budějovicemi hladce 3:0.\n\nOstrava deklasovala Příbram podruhé v sezoně a o branku předčila výhru 4:0 z podzimního vzájemného duelu.\n\nBaník o vítězství rozhodl slepenými góly Gigliho Ndefeho a Daniela Holzera v rozmezí dvou minut v závěru první půle. Po přestávce se prosadili ještě Daniel Tetour, Ondřej Šašinka a Jakub Pokorný.\n\nOstrava v druhém utkání po odvolání trenéra Luboše Kozla a v domácí premiéře pod Ondřejem Smetanou zaznamenala nejvyšší ligové vítězství od 30. dubna 2006. Příbram už 12 kol nevyhrála a v posledních čtyřech nebodovala.</text>\n<people>\n <person id="3282" name="Gigli Ndefe">\n  <role>fotbalo

In [63]:
def transform_event(e, id_):
    te = {"@id": int(id_), "@text": e["event"]}
    return te

def transform_event_full(e, id_):
    # include subevents
    te = {"@id": int(id_), "@text": e["event"]}
    subevents = e["subevents"]
    if len(subevents) > 0:
        ses = []
        for idx, se in enumerate(subevents):
            ses.append({"@subid": idx+1, "@text": se["event"]})
        te["subevents"] = ses
            
    return te
    
    
def prepare_query(events, transform_fn=transform_event):
    events = deepcopy(events)
    xml_string = re.sub(r' id=(\d+)', r' id="\1"', events["query"])
    root = xmltodict.parse(f'<root>{xml_string}</root>')
    x = root["root"]
    
    # The Deepseek model had a hard time with decoding the original Czech formatted dates
    locale.setlocale(locale.LC_TIME, "cs_CZ.UTF-8") 
    original_format = "%a, %d %b %Y %H:%M:%S"
    new_format = "%A, %d %B %Y (%Y-%m-%d) %H:%M:%S"
    parsed_date = datetime.strptime(x["date"], original_format)
    formatted_date = parsed_date.strftime(new_format)

    x["date"] = formatted_date
    
    equery = {'event': [transform_fn(e, idx+1) for idx, e in enumerate(events["answer"])]}
    x["events"] = equery
    
    if "people" in x:
        del x["people"]
    if "organizations" in x:
        del x["organizations"]
    if "locations" in x:
        del x["locations"]
    
    query = xmltodict.unparse(root, pretty=True, indent="|")
    # now cut root and fix formatting
    query = '\n'.join(line[1:].replace('|', ' ') if line.startswith('|') else line for line in query.splitlines()[2:-1])

    return query

idx = 10
ret = prepare_query(eventsV4[idx], transform_fn=transform_event_full)
print(ret)

<date>Sobota, 13 března 2021 (2021-03-13) 19:18:00</date>
<text>Fotbalisté Baníku rozstříleli Příbram 5:0, Opava porazila gólem z penalty Pardubice

Fotbalisté Příbrami utrpěli ve 23. kole první ligy debakl 0:5 v Ostravě a na posledním místě tabulky ztrácejí už tři body na předposlední Opavu. Slezané totiž porazili nováčka z Pardubic 1:0. Zlín zvítězil nad Českými Budějovicemi hladce 3:0.

Ostrava deklasovala Příbram podruhé v sezoně a o branku předčila výhru 4:0 z podzimního vzájemného duelu.

Baník o vítězství rozhodl slepenými góly Gigliho Ndefeho a Daniela Holzera v rozmezí dvou minut v závěru první půle. Po přestávce se prosadili ještě Daniel Tetour, Ondřej Šašinka a Jakub Pokorný.

Ostrava v druhém utkání po odvolání trenéra Luboše Kozla a v domácí premiéře pod Ondřejem Smetanou zaznamenala nejvyšší ligové vítězství od 30. dubna 2006. Příbram už 12 kol nevyhrála a v posledních čtyřech nebodovala.</text>
<events>
 <event id="1" text="Baník Ostrava porazil Příbram 5:0.">
  <subeven

In [29]:
write_jsonl("data/labeled_datasets/event_temp_val/V1/bootstrap_queries.jsonl", [{"query": prepare_query(ev), "answer": ""} for ev in eventsV4])

In [ ]:
# now create answers by combining predictions with original events
def combine_with_predictions(events, preds):
    assert len(events) == len(preds)
    events = deepcopy(events)
    for evnt, pred in zip(events, preds):
        evnt["query"] = pred["query"]
        src_event = evnt["answer"]
        dst_event = pred["pred"]
        assert len(src_event) == len(dst_event)
        new_events = []
        for idx, (se, de) in enumerate(zip(src_event, dst_event)):
            se = deepcopy(se)
            assert de["id"] == idx+1, (de["id"], idx+1)
            se["time_start"] = de["time_start"]
            se["time_end"] = de["time_end"]
            se["time_reported"] = de["time_reported"]
            new_events.append(se)
        evnt["answer"] = new_events
    write_jsonl("data/labeled_datasets/event_temp_val/V1/event_temp_val-V1_to_fix.jsonl", events)
    
preds = read_jsonl("data/labeled_datasets/event_temp_val/V1/predicted_temp_val.jsonl")
combine_with_predictions(eventsV4, preds)

### V1
Based on bootstrapped

In [60]:
event_temp_val_boot = read_jsonl("data/labeled_datasets/event_temp_val/V1/event_temp_val-BOOT_fixed.jsonl")

In [64]:
def convert_fixed_boot_to_V1(data, out_file):
    data = deepcopy(data)
    for idx, sample in enumerate(data):
        # print(idx)
        sample["query"] = prepare_query(sample, transform_fn=transform_event_full)
        # print(sample["query"])
        new_events = []
        for idx, ev in enumerate(sample["answer"]):
            subevents = ev["subevents"]
            new_subevents = []
            for sidx, sev in enumerate(subevents):
                new_subevents.append(
                    {
                        "subid": sidx + 1,
                        "time_start": sev.get("time_start", "PARENT"),
                        "time_end": sev.get("time_end", "PARENT"),
                    }
                )
            nev = {
                "id": idx + 1,
                "subevents": new_subevents,
                "time_start": ev["time_start"],
                "time_end": ev["time_end"],
                "time_reported": ev["time_reported"],
            }
            new_events.append(nev)
        sample["answer"] = new_events
    write_jsonl(out_file, data)
    return data


data2 = convert_fixed_boot_to_V1(event_temp_val_boot, "data/labeled_datasets/event_temp_val_V1.jsonl")

## Events to Canonical (FULL) CRO dataset


### V1

The CRO dataset is now convoluted: part of the input (e.g., date, text, NERs) are encoded in query. Let us prepare JSONL-only canonical version called **full** which contains all information and can be used by newly created *pipeline* ops to be converted to any format. This first version is taken from the newest data I have: Events V4 and Event Temporal Validity V1.

In [75]:
def create_cro_full_V1():
    events = read_jsonl("data/labeled_datasets/cro/events/V4/events_V4_fixed.jsonl")
    tvalidity = read_jsonl("data/labeled_datasets/event_temp_val_V1.jsonl")
    attribs = read_jsonl("data/labeled_datasets/events_V3_attributions.jsonl") # only to get cro_idx
    assert len(events) == len(tvalidity)
    samples = []
    for ev, tv, at in zip(events, tvalidity, attribs):
        assert ev["id"] == tv["id"]
        # print(ev.keys())
        # print(tv.keys())
        xml_string = re.sub(r' id=(\d+)', r' id="\1"', ev["query"])
        root = xmltodict.parse(f'<root>{xml_string}</root>')
        evroot = root["root"]
        
        xml_string = re.sub(r' id=(\d+)', r' id="\1"', tv["query"])
        root = xmltodict.parse(f'<root>{xml_string}</root>')
        tvroot = root["root"]
        
        xml_string = re.sub(r' id=(\d+)', r' id="\1"', at["query"])
        root = xmltodict.parse(f'<root>{xml_string}</root>')
        atroot = root["root"]
        
        assert len(evroot["text"]) == len(tvroot["text"]) == len(atroot["text"])

        def aslist(e):
            if e:
                return e if isinstance(e, list) else [e]
            else:
                return []
            
        people_recs = aslist(evroot["people"]["person"]) if evroot["people"] else []
        locations_recs = aslist(evroot["locations"]["loc"]) if evroot["locations"] else []
        organizations_recs = aslist(evroot["organizations"]["org"]) if evroot["organizations"] else []
        
        people = [{"id": e["@id"], "name": e["@name"], "roles": aslist(e.get("role"))} for e in people_recs]
        locations = [{"id": e["@id"], "name": e["@name"], "abbreviation": e["@abbreviation"], "type": e["@type"]} for e in locations_recs]
        organizations = [{"id": e["@id"], "name": e["@name"], "abbreviation": e["@abbreviation"], "type": e["@type"]} for e in organizations_recs]
        # pprint(evroot["organizations"])
        
        assert len(ev["answer"]) == len(tv["answer"])
        events = []
        for ev_id, (eva, tva) in enumerate(zip(ev["answer"], tv["answer"]), start=1):
            event = deepcopy(eva)
            event["time_start"] = tva["time_start"]
            event["time_end"] = tva["time_end"]
            event["time_reported"] = tva["time_reported"]
            assert len(event["subevents"]) == len(tva["subevents"])
            for sevent, stva in zip(event["subevents"], tva["subevents"]):
                sevent["time_start"] = stva["time_start"]
                sevent["time_end"] = stva["time_end"]
            for idx in range(len(event["subevents"])):
                event["subevents"][idx] = {"subid": idx+1, **event["subevents"][idx]}
            events.append(event)
        for idx in range(len(events)):
            events[idx] = {"id": idx+1, **events[idx]}
        
        samples.append({"id": ev["id"], "cro_idx": at["cro_idx"], "date": tvroot["date"], "text": tvroot["text"], 
                        "people": people, "locations": locations, "organizations": organizations,
                        "events": events})
    write_jsonl("data/labeled_datasets/cro/full/cro_full_V1.jsonl", samples)
    
    
create_cro_full_V1()

## Merge Tereza's evaluation notes

In [7]:
def merge_notes(data_fname, notes_fname, out_fname):
    data = read_jsonl(data_fname)  # pipeline export
    notes = read_jsonl(notes_fname)  # includes notes by Tereza
    assert len(data) == len(notes)
    for d, n in zip(data, notes):
        # assert d["query"] == n["query"]
        events1 = d["answer"]["events"]
        events2 = n["answer"]
        for e1, e2 in zip(events1, events2):
            print(pf(e1))
            print(pf(e2))
            assert e1["event"] == e2["event"]
            assert e1["people"] == e2["people"]
            assert e1["orgs"] == e2["orgs"]
            assert e1["locations"] == e2["locations"]
            for s1, s2 in zip(e1["subevents"], e2["subevents"]):
                if "event" not in s2:
                    assert s1["event"] == s2["events"]  # error in teresa's data
                    # print(pf(s1))
                    # print(pf(s2))
                else:
                    assert s1["event"] == s2["event"]

            if "note" in e2:
                e1["note"] = e2["note"]

            # break
        # print(n["query"])
        # break
        write_jsonl(out_fname, data)

In [ ]:
merge_notes(
    data_fname="data/labeled_datasets/cro/partial/V1/cro_all_V1.jsonl",
    notes_fname="data/labeled_datasets/cro/partial/V1/dataset_tereza_V1.jsonl",
    out_fname="data/labeled_datasets/cro/partial/V1/dataset_events_wtih_time_V1.jsonl",
)

## Fix IDs to numbers

Some dataset files have string ids instead of number ids.

In [ ]:
def fix_dataset_ids(fname, format="query"):
    data = read_jsonl(fname)
    for d in data:
        for section in ["people", "organizations", "locations"]:
            sec = d["answer"][section] if format == "query" else d[section]
            for rec in sec:
                rec["id"] = int(rec["id"])
    write_jsonl(fname, data)
    
fix_dataset_ids("data/labeled_datasets/cro/partial/V1/cro_all_V1.jsonl")
fix_dataset_ids("data/labeled_datasets/cro/partial/V1/cro_full_V1.jsonl")
fix_dataset_ids("data/labeled_datasets/cro/partial/V1/dataset_tereza_V2.jsonl")
fix_dataset_ids("data/labeled_datasets/cro/full/cro_full_V1.jsonl", format="dataset")

## Convert Tereza's evaluation (with time) to new format

In [ ]:
def convert_notes(data_fname, notes_fname, out_fname, schema_fname):
    data = read_jsonl(data_fname)  # pipeline export
    notes = read_jsonl(notes_fname)  # includes notes by Tereza
    schema = json.loads(Path(schema_fname).read_text())
    print(len(data), len(notes))
    assert len(data) == len(notes)
    
    ret = []
    for d, n in zip(data, notes):
        e = {"id": d["id"], "cro_idx": d["cro_idx"]}
        # for section in ["people", "organizations", "locations"]:
        #     for rec in n["answer"][section]:
        #         rec["id"] = int(rec["id"])
        e.update(n["answer"])
        validate(e,  schema=schema) 
        ret.append(e)
        
    
    write_jsonl(out_fname, ret)
        
convert_notes(
    data_fname="data/labeled_datasets/cro/partial/V1/cro_all_V1.jsonl",
    notes_fname="data/labeled_datasets/cro/partial/V1/dataset_tereza_V2.jsonl",
    out_fname="data/labeled_datasets/cro/partial/V1/dataset_events_wtih_time_V2.jsonl",
    schema_fname="data/schemas/schema_cro_full_strict.json"
)

30 30


## Unlabeled and Inference Samples

Generate samples to test overall pipeline

In [3]:
def export_samples(txts, out_file, start, stop):
    original_format = "%a, %d %b %Y %H:%M:%S"
    new_format = "%A, %d %B %Y (%Y-%m-%d) %H:%M:%S"
    samples = []
    for idx, txt in enumerate(txts[start:stop], start=1):
        date_ = txt.splitlines()[0]
        parsed_date = datetime.strptime(date_, original_format)
        formatted_date = parsed_date.strftime(new_format)
    
        txt = ("\n".join(txt.splitlines()[2:])).strip()
        sample = {"id": idx, "cro_idx": start+idx-1, "date": formatted_date, "text": txt}
        samples.append(sample)
    write_jsonl(out_file, samples)

In [5]:
n_pars = 5 # same as used to optimize prompts
recs, kws = cro_data.import_cro("/mnt/data/cro/factcheck/v1/interim/cro_all.jsonl", prepend_date=True)
txts = get_initial_spans(recs, n_pars=n_pars)
print(len(txts))
export_samples(txts, f"data/labeled_datasets/cro/full/cro_unlabeled_V2.jsonl", start=5000, stop=10000)

104482


In [ ]:
n_pars = 5 # same as used to optimize prompts
recs, kws = cro_data.import_cro("/mnt/data/cro/factcheck/v1/interim/cro_all.jsonl", prepend_date=True)
txts = get_initial_spans(recs, n_pars=n_pars)
print(len(txts))
export_samples(txts, f"data/extraction_pipeline/cro_data_npars{n_pars}.jsonl", start=10000, stop=11000)

In [69]:
n_pars = 10 # longer
recs, kws = cro_data.import_cro("/mnt/data/cro/factcheck/v1/interim/cro_all.jsonl", prepend_date=True)
txts = get_initial_spans(recs, n_pars=n_pars)
print(len(txts))
export_samples(txts, f"data/extraction_pipeline/cro_data_npars{n_pars}.jsonl", start=10000, stop=11000)

73143


In [4]:
n_pars = 5
recs, kws = cro_data.import_cro("/mnt/data/cro/factcheck/v1/interim/cro_all.jsonl", prepend_date=True)
txts = get_initial_spans(recs, n_pars=n_pars, max_pars="all")
print(len(txts))
export_samples(txts, f"data/extraction_pipeline/cro_data_npars-min{n_pars}.jsonl", start=10000, stop=11000)

104482


# SiR 1.0

In [2]:
def import_brat_annotation(sir_path, doc_id):
    txt = Path(sir_path, f"{doc_id}.txt").read_text()
    ann = Path(sir_path, f"{doc_id}.ann").read_text()
    
    entities = {}
    relations = {}
    
    pattern_entity = re.compile(r'(\S+)\s+(\S+)\s+(\d+)\s+(\d+)\s+(.+)')
    for line in ann.splitlines():
        if line.startswith("R"):
            id_, type_, arg1, arg2 = line.split()
            assert type_ == "attribution", type_
            arg1 = arg1.split(":")[1]
            arg2 = arg2.split(":")[1]
            relations[id_] = {"arg1": arg1, "arg2": arg2}
        else:
            match = pattern_entity.match(line)
            if match:
                id_, type_, sidx, eidx, span = match.groups()
                sidx = int(sidx)
                eidx = int(eidx)
                assert span == txt[sidx:eidx], (span, txt[sidx:eidx])
                entities[id_] = {"type": type_, "start": sidx, "end": eidx, "span": span}
            else:
                assert False, line
                
    # order by start index and rename the entities
    ents = list(entities.items())
    ents.sort(key=lambda e: e[1]["start"])
    id2newid = {e[0]: f"T{i+1}" for i, e in enumerate(ents)}
    entities = [{"id": id2newid[e[0]], **e[1]} for e in ents]
    
    # fix entity ids in relations
    for r in relations.values():
        r["arg1"] = id2newid[r["arg1"]]
        r["arg2"] = id2newid[r["arg2"]]
        
    # now sort relations by their first argument
    rels = list(relations.items())
    rels.sort(key=lambda e: int(e[1]["arg1"][1:]))
    relations = [{"id": f"R{i+1}", **e[1]} for i, e in enumerate(rels)]
    
    return {"query": txt, "answer": {"entities": entities, "relations": relations}}

# import_brat_annotation("/mnt/data/factcheck/ufal/SiR1.0/data/triple_manual", "doc-5457987.xml")
import_brat_annotation("/mnt/data/factcheck/ufal/SiR1.0/data/triple_manual", "doc-5753090.xml")

{'query': 'Veslaři si pochvalují domácí atmosféru na mistrovství Evropy v Račicích \nSnad jen na domácím šampionátu zažijí veslaři něco podobného. \nNa startu jim organizátoři přejí mnoho štěstí v češtině a v zádech mají velkou podporu domácího publika. \nTu snad využijí v sobotním programu skifař Ondřej Synek a dvojka bez kormidelníka pro postup do finále. \nMyslím si, že podpora bude veliká. \nCo jsem slyšel, tak lístků už se prodalo dost. \nSnad to bude dobrý," řekl Radiožurnálu Jakub Podrazil z dvojky bez kormidelníka, \nPrávě česká dvojka bez kormidelníka bude dnes podporu domácích fanoušků potřebovat. \nV sobotu dopoledne pojede v opravě o postup do finále. \nNa mě to tady dýchá z každé strany. \nProtože když jedu na start, tak tam všichni mluví česky. \nBójkář, co mi drží loď, na mě mluví česky a přeje mi štěstí. \nTohle se stává jenom na mistrovství republiky, ne na takových závodech," komentuje výhodu domácí atmosféry nejlepší český skifař Ondřej Synek \nBojkář mi říkal, že mi

In [3]:
def select_and_rearragne_entity_types(data, must_contain, rng):
    new_data = []
    for e in data:
        entities = e["answer"]["entities"]
        types = [ent["type"] for ent in entities]
        types = set(types)
        
        select = 0
        for c in must_contain:
            if c in types:
                select += 1
        
        for t in types:
            if t not in must_contain and t != "PHRASE":
                select = 0
                
        if "PHRASE" not in types:
            select = 0
            
        if select > 0:
            types = list(types)
            types.sort()
            # print(i, select, types)
            new_data.append((select, e))
            rng.shuffle(new_data)
            new_data.sort(key=lambda e: e[0], reverse=True)
    new_data = [e[1] for e in new_data]
    return new_data


def import_sir(sir_path, rng, relations=True, spans=True, ids=True):
    data = []
    for fname in Path(sir_path).glob('*.ann'):
        doc_id = fname.name[:-4]
        ex = import_brat_annotation(sir_path, doc_id)
        if not spans:
            ents = ex["answer"]["entities"]
            for e in ents:
                del e["start"]
                del e["end"]
        if not ids:
            ents = ex["answer"]["entities"]
            for e in ents:
                del e["id"]
        if not relations:
            del ex["answer"]["relations"]
        data.append(ex)
        
    # this rearanges the samples so samples with most hits in must_contain_samples are first
    # sort-of stratification
    # for triple_manual the first roughly 20 samples cover nicely 'official-non-political', 'official-political', 'anonymous-partial'
    # there must be at least one PHRASE in all samples
    data = select_and_rearragne_entity_types(data,
                                             must_contain=['official-non-political', 'official-political', 'anonymous-partial'],
                                             rng=rng)
    
    return data
    

# subset = "single"
# subset = "double_unified"
subset = "triple_manual"
data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234))
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}.jsonl", data)

data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234), relations=False)
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations.jsonl", data)

data = import_sir(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", rng=np.random.RandomState(1234), relations=False, spans=False, ids=False)
write_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations_no_spans_no_ids.jsonl", data)



In [4]:
data = read_jsonl(f"/home/drchajan/devel/python/FC/long_sum/data/labeled_datasets/sir1.0_{subset}_no_relations_no_spans_no_ids.jsonl")

In [6]:
schema = {
  "type": "object",
  "properties": {
    "entities": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": { "type": "string"},
          "type": { "type": "string"},
          "start": { "type": "integer"},
          "end": { "type": "integer"},
          "span": { "type": "string"}
        },
        "required": ["id", "type", "start", "end", "span"]
      }
    },
    "relations": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": { "type": "string"},
          "arg1": { "type": "string" },
          "arg2": { "type": "string" }
        },
        "required": ["id", "arg1", "arg2"]
      }
    }
  },
  "required": ["entities", "relations"],
  "additionalProperties": False
}

## Just phrases

In [10]:
def import_sir_phrases(sir_path, rng, sorted_=True, encapsulate=False):
    data = []
    for fname in Path(sir_path).glob('*.ann'):
        doc_id = fname.name[:-4]
        ex = import_brat_annotation(sir_path, doc_id)
        entities = ex["answer"]["entities"]
        phrases = set()
        for e in entities:
            if e["type"] == "PHRASE":
                phrases.add(e["span"])
        phrases = list(phrases)
        if sorted_:
             phrases = sorted(phrases)
        if encapsulate:
            phrases = {"array": phrases}
        data.append({"query": ex["query"], "answer": phrases})
    data = rng.permutation(data)
    return data
        
subset = "triple_manual"

data = import_sir_phrases(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", np.random.RandomState(1234))
write_jsonl(f"data/labeled_datasets/sir1.0_{subset}_phrases.jsonl", data)

# this format is needed for OpenAI models which cannot deal with JSON arrays
data = import_sir_phrases(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", np.random.RandomState(1234), encapsulate=True)
write_jsonl(f"data/labeled_datasets/sir1.0_{subset}_phrases_enc.jsonl", data)


In [11]:
data[0]

{'query': 'Zájmové, pohybové, ale i vzdělávací aktivity. \nŽákům mají o prázdninách pomoci doučovací kempy \nDohnat učivo, zmenšit vědomostní rozdíly mezi žáky způsobené v době koronaviru výukou na dálku a taky děti zapojit třeba do sportovních aktivit. \nPrávě s tím by měly žákům pomoct letní doučovací kempy. \nMinisterstvo školství na ně vyčlenilo celkem 100 milionů korun. \nResort tak naváže na loňský prázdninový projekt Vzdělávací dny. \nOba jsou dis. \nMají poruchu čtení, psaní a pravopisu, takže vyhledávám tyto aktivity i v takové hravé formě, říká paní Lenka Radiožurnálu. \nSvé dva syny, devítiletého Ondřeje a desetiletého Martina, přihlásila Lenka už loni na vzdělávací dny do Střediska volného času v Holešově. \nJako velmi důležité zmiňuje zařazení pohybových aktivit do programu. \nA myslím si, že letos to bude úplně extrémně důležité, protože pohyb není vůbec, zdůrazňuje Lenka. \nStředisko volného času TYMY Holešov loni uspořádalo celkem 12 vzdělávacích dnů, kterých se zúčastn

In [7]:
print(data[0]["query"])

Zájmové, pohybové, ale i vzdělávací aktivity. 
Žákům mají o prázdninách pomoci doučovací kempy 
Dohnat učivo, zmenšit vědomostní rozdíly mezi žáky způsobené v době koronaviru výukou na dálku a taky děti zapojit třeba do sportovních aktivit. 
Právě s tím by měly žákům pomoct letní doučovací kempy. 
Ministerstvo školství na ně vyčlenilo celkem 100 milionů korun. 
Resort tak naváže na loňský prázdninový projekt Vzdělávací dny. 
Oba jsou dis. 
Mají poruchu čtení, psaní a pravopisu, takže vyhledávám tyto aktivity i v takové hravé formě, říká paní Lenka Radiožurnálu. 
Své dva syny, devítiletého Ondřeje a desetiletého Martina, přihlásila Lenka už loni na vzdělávací dny do Střediska volného času v Holešově. 
Jako velmi důležité zmiňuje zařazení pohybových aktivit do programu. 
A myslím si, že letos to bude úplně extrémně důležité, protože pohyb není vůbec, zdůrazňuje Lenka. 
Středisko volného času TYMY Holešov loni uspořádalo celkem 12 vzdělávacích dnů, kterých se zúčastnilo 180 dětí ve věku o

In [20]:
for i, e in enumerate(data):
    print(i, e["answer"])

0 ['podle', 'popisuje', 'upozorňuje', 'zdůrazňuje', 'zmiňuje', 'říká']
1 ['dodala', 'připomíná', 'reagují', 'upozorňují', 'uvedla', 'řekla']
2 ['podle', 'varovali', 'zveřejnily']
3 ['konstatovali', 'prohlásila', 'se domnívá']
4 ['dodala', 'kritizoval', 'mluvil', 'nabídli', 'řekl', 'říkal']
5 ['Vyplývá']
6 ['Uvedla', 'podle']
7 ['informova', 'napsal', 'podle', 'prohlásil', 'připustil', 'řekl']
8 ['Informovala', 'Podle', 'napsal', 'se ví']
9 ['Myslím si', 'jsem slyšel', 'komentuje', 'nepřeceňuje', 'reprodukuje', 'řekl']
10 ['Dodala', 'Oznámila', 'informovala', 'oznámilo', 'píše', 'uvedla']
11 ['Podle']
12 ['Podle', 'Prý', 'zpochybnil', 'řekl']
13 ['Podle', 'říká']
14 ['popisuje', 'prohlásil', 'upozorňuje', 'zdůrazňuje', 'říká']
15 ['podle', 'prohlásil']
16 []
17 ['nazývá', 'upozorňuje']
18 ['informoval', 'uvedl', 'řekl']
19 ['podle', 'řekl']
20 ['informova', 'uvedl']
21 ['Podle', 'dodal', 'informoval', 'řekl', 'žertoval']
22 []
23 ['doporučuje', 'nabádá', 'píše', 'stojí']
24 ['Podle', 'p

In [9]:
[1, 2, 3].index(2)

1

In [15]:
def import_sir_phrases_occurences(sir_path, rng):
    def find_all_offsets(text, substring):
        offsets = []
        start = 0
        
        while True:
            # Find the next occurrence of the substring
            start = text.find(substring, start)
            if start == -1:
                break  # No more occurrences found
            offsets.append(start)  # Add the index to the list
            start += 1  # Move to the next character to avoid finding the same substring again
        
        return offsets
        
    data = []
    for fname in Path(sir_path).glob('*.ann'):
        doc_id = fname.name[:-4]
        ex = import_brat_annotation(sir_path, doc_id)
        entities = ex["answer"]["entities"]
        query = ex["query"]
        phrases = []
        for e in entities:
            if e["type"] == "PHRASE":
                span = e["span"]
                offsets = find_all_offsets(query, span)
                occurence = offsets.index(e["start"]) + 1
                phrases.append({"span": span, "occurence": occurence})
        phrases = list(phrases)
        data.append({"query": query, "answer": phrases})
    data = rng.permutation(data)
    return data
        
subset = "triple_manual"

data = import_sir_phrases_occurences(f"/mnt/data/factcheck/ufal/SiR1.0/data/{subset}", np.random.RandomState(1234))
write_jsonl(f"data/labeled_datasets/sir1.0_{subset}_phrases_occurences.jsonl", data)


In [16]:
data[0]

{'query': 'Zájmové, pohybové, ale i vzdělávací aktivity. \nŽákům mají o prázdninách pomoci doučovací kempy \nDohnat učivo, zmenšit vědomostní rozdíly mezi žáky způsobené v době koronaviru výukou na dálku a taky děti zapojit třeba do sportovních aktivit. \nPrávě s tím by měly žákům pomoct letní doučovací kempy. \nMinisterstvo školství na ně vyčlenilo celkem 100 milionů korun. \nResort tak naváže na loňský prázdninový projekt Vzdělávací dny. \nOba jsou dis. \nMají poruchu čtení, psaní a pravopisu, takže vyhledávám tyto aktivity i v takové hravé formě, říká paní Lenka Radiožurnálu. \nSvé dva syny, devítiletého Ondřeje a desetiletého Martina, přihlásila Lenka už loni na vzdělávací dny do Střediska volného času v Holešově. \nJako velmi důležité zmiňuje zařazení pohybových aktivit do programu. \nA myslím si, že letos to bude úplně extrémně důležité, protože pohyb není vůbec, zdůrazňuje Lenka. \nStředisko volného času TYMY Holešov loni uspořádalo celkem 12 vzdělávacích dnů, kterých se zúčastn